In [ ]:
# Script to gather VASP input files and modify parameters for KPOINTS and INCAR
# Requires locally saved POSCAR file, and reference KPOINTS and INCAR files
# POTCAR file is generated by appending POTCAR files (on server) for each element
# Server side .sbatch file is utilized

In [2]:
from pyExecVaspSlurm import Job, Server

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
# Initialize Job
job = Job('MD_ML_Test_MRB2_12_550eV_3x5x6_bulk_4c',poscar_path_local='/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/Ex_4b_poscar_beta_LPS')

job.incar_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/incar_MD_ML')
job.kpoints_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/kpoints_reference')

MD_ML_Test_MRB2_12_550eV_3x5x6_bulk_4c Job instance started


In [5]:
job.elements

array(['S', 'Li', 'P'], dtype='<U2')

In [51]:
# Modify INCAR and KPOINTS
job.incar_modify('ENCUT','550')
job.incar_modify('ISMEAR','0') # Gaussian smearing
job.incar_modify('SIGMA','0.2')
job.incar_modify('PMASS','1000')
job.kpoints_modify('subdivisions along recipr. l.', '3 5 6')

In [6]:
# Initialize Server
# batch-file for shared memory (ipcs & system-V): /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_sysv.sbatch
# batch-file for MPI-3 shared-memory capabilities: /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_shmem.sbatch
# ipcs shared memory and MPI 2021.9.0. : /home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1_sysv_mpi2021.9.0.sbatch
server = Server('st_st179390','justus2.uni-ulm.de', '/lustre/home/st/st_us-031400/st_st179390', '/home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1.sbatch')
sftp, ts = server.connect()

Your OTP: Password: <paramiko.Transport at 0x36b8b0d0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>


In [114]:
# Use real space projection operators with automatic optimization

#add_line = pd.DataFrame({
#    'Setting': ['LREAL'],
#    'Value': ['Auto']
#})

#job.incar = pd.concat([job.incar, add_line])

In [27]:
job.kpoints

,Value,Description
0,Automatic mesh,descriptive line
1,0,k-points (0: automatic generation)
2,Gamma,Gamma centered grid
3,3 5 6,subdivisions along recipr. l.
4,0. 0. 0.,optional shift of the mesh


In [58]:
job.incar

,Setting,Value
0,ISTART,0
1,ISPIN,1
2,GGA,PE
3,ISMEAR,0
4,SIGMA,0.2
5,ENCUT,550
6,PREC,accurate
7,ADDGRID,.TRUE.
8,EDIFF,1.0E-7
9,EDIFFG,-5.0E-3


In [57]:
job.incar_modify('ML_LMLFF','.TRUE.')

In [59]:
job.name = 'ML_Test_on_3x5x6_550eV_bulk_4b'

In [60]:
server.create_folder(sftp, job.name)
server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.poscar_upload(job,sftp)
server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.name_batch(job, ts)
server.batch_submit(job, ts)

Created POTCAR for S
Appended POTCAR for Li
Appended POTCAR for P
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0


In [33]:
#upload and start jobs for RCUT1 & 2 evaluation

job.incar_modify('ML_LMLFF','.TRUE.')
parameters = pd.DataFrame({
    'ML_RCUT1': [8.0, 10.5, 13.0],
    'ML_RCUT2': [5.0, 7.5, 10.0]
})

for rcut1 in parameters['ML_RCUT1']:
    for rcut2 in parameters['ML_RCUT2']:
        job.incar_modify('ML_RCUT1',rcut1)
        job.incar_modify('ML_RCUT2',rcut2)

        k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
        e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
        job.name = f'ML_Test_RCUT1_{rcut1}A_RCUT2_{rcut2}A_{k_points}_{int(e_cut)}eV_bulk_4b'
        print(job.name)

        #server.create_folder(sftp, job.name)
        #server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
        #server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
        #server.poscar_upload(job,sftp)
        #server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
        server.name_batch(job, ts)
        server.batch_submit(job, ts)

ML_Test_RCUT1_8.0A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_8.0A_RCUT2_7.5A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_8.0A_RCUT2_10.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_7.5A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_10.5A_RCUT2_10.0A_3x5x6_550eV_bulk_4b
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
ML_Test_RCUT1_13.0A_RCUT2_5.0A_3x5x6_550eV_bulk_4b
Up

In [16]:
server.name_batch(job, ts)

Updating name - exit status of execution command: 0


In [17]:
# upload and start MD jobs for PMASS evaluation

parameters = pd.DataFrame({
    'PMASS': ['500', '750', '1000', '1250', '1500'],
})


for pmass in parameters['PMASS']:
    job.incar_modify('PMASS',pmass)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'MD_Test_PMASS_{pmass}amu_{k_points}_{int(e_cut)}eV_bulk_4b'
    print(job.name, pmass)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts)
    server.batch_submit(job, ts)

MD_Test_PMASS_500amu_3x5x6_460eV_bulk_4b 500
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_PMASS_750amu_3x5x6_460eV_bulk_4b 750
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_PMASS_1000amu_3x5x6_460eV_bulk_4b 1000
Created POTCAR for 

In [9]:
# upload and start MD jobs for TEBEG evaluation

parameters = pd.DataFrame({
    'TEBEG': ['900', '1100', '1200', '850'],
})


for tebeg in parameters['TEBEG']:
    job.incar_modify('TEBEG',tebeg)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'MD_Test_TEBEG_{tebeg}k_{k_points}_{int(e_cut)}eV_bulk_4b'
    print(job.name, tebeg)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts)
    server.batch_submit(job, ts)

MD_Test_TEBEG_900k_3x5x6_460eV_bulk_4b 900
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_TEBEG_1100k_3x5x6_460eV_bulk_4b 1100
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
MD_Test_TEBEG_1200k_3x5x6_460eV_bulk_4b 1200
Created POTCAR for Li
A

In [117]:
# upload and start jobs for mesh evaluation
for mesh in parameters['k-points']:
    job.kpoints_modify('subdivisions along recipr. l.', mesh)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'bulk_LPS_LREAL_auto_mesh_{k_points}_{int(e_cut)}eV'
    print(k_points)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.batch_submit(job, ts)

1x1x1
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
2x2x2
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
2x2x4
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
1x2x2
Created POTCAR for Li
Appended POTCAR for P
Ap

In [46]:
# upload and start jobs for energy cutoff evaluation
for ecut in parameters['Energy cutoff [eV]']:
    job.incar_modify('ENCUT', str(ecut))
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    job.name = f'bulk_LPS_{k_points}_{int(ecut)}eV'

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.batch_submit(job, ts)


Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0
Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
Temporarirly saved INCAR removed
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0


In [37]:
# create and start single job

server.create_folder(sftp, job.name)
server.potcar_gather(job, sftp, '/lustre/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.poscar_upload(job, sftp)
server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.batch_submit(job, ts)

Created POTCAR for Li
Appended POTCAR for P
Appended POTCAR for S
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Exit status of execution command: 0


In [14]:
# Terminate connection
if sftp: sftp.close()
if ts.active: ts.close()